In [3]:
from dotenv import load_dotenv
load_dotenv()

import os
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")


In [ ]:
from langchain_openai import ChatOpenAI

gptLlm = ChatOpenAI(model_name="o3-mini")

gptLlm.invoke("What is the capital of France?")

AIMessage(content='The capital of France is Paris.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 13, 'total_tokens': 98, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'o3-mini-2025-01-31', 'system_fingerprint': 'fp_4cd4f3e64a', 'id': 'chatcmpl-CmJshk8YkD69tIOKKgSplBfzVq1Fz', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b17e1-5584-7bb0-aa74-c764c6e65217-0', usage_metadata={'input_tokens': 13, 'output_tokens': 85, 'total_tokens': 98, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 64}})

In [10]:
from langchain_groq import ChatGroq

llm = ChatGroq(model_name="llama-3.3-70b-versatile")

llm.invoke("My name is Akkshay.")


AIMessage(content="Hello Akkshay, it's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 43, 'total_tokens': 72, 'completion_time': 0.0486406, 'completion_tokens_details': None, 'prompt_time': 0.001188106, 'prompt_tokens_details': None, 'queue_time': 0.052873114, 'total_time': 0.049828706}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b17ef-9869-7013-9775-6f71ce3b4e0e-0', usage_metadata={'input_tokens': 43, 'output_tokens': 29, 'total_tokens': 72})

In [13]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geographical facts assistant."),
    ("human", "{input}"),
])

chain = prompt | llm

res = chain.invoke({"input": "What is the capital of helsinki?"})
print(res.content)


Helsinki is actually the capital of Finland, not the other way around. So, Helsinki does not have a capital. It is the largest city and the capital of Finland, a country located in Northern Europe.


In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

gptLlm = ChatOpenAI(model_name="o3-mini")

gptLlm.invoke("What is the capital of France?")

parser = StrOutputParser()

chain = prompt | gptLlm | parser

chain.invoke({"input": "Give me 10 facts about finland"})


'1. Finland is a Northern European country located in the Nordic region, bordered by Sweden to the west, Russia to the east, and Norway to the north, with the Baltic Sea to the south.\n2. Known as the "Land of a Thousand Lakes," Finland actually boasts over 188,000 lakes, making water a central part of its geography and lifestyle.\n3. The Finnish education system is highly regarded worldwide for its quality, equality, and innovative teaching methods, consistently ranking among the best.\n4. Helsinki, the capital, is not only the largest city in Finland but also a hub for design, architecture, and cultural activities, reflecting the country\'s modern yet nature-connected lifestyle.\n5. Finland experiences dramatic seasonal changes, including long, dark winters with the phenomenon of the "Polar Night" and bright summers featuring the "Midnight Sun."\n6. The country is a parliamentary republic, with a president serving as head of state and a prime minister leading the government.\n7. Finn

In [25]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

parser = JsonOutputParser()

prompt = PromptTemplate(
    template="You are a geographical facts assistant. Answer in valid JSON. {format_instruction}\n\nQuestion: {input}",
    input_variables=["input"],
    partial_variables={"format_instruction": parser.get_format_instructions()}
)

chain = prompt | gptLlm | parser

try:
    result = chain.invoke({"input": "finland"})
    print("Result:", result)
except Exception as e:
    print("Error invoking chain:", e)
    result = None



Result: {'country': 'Finland', 'officialName': 'Republic of Finland', 'capital': 'Helsinki', 'region': 'Northern Europe', 'population': 'Approximately 5.5 million', 'area_km2': 338424, 'officialLanguages': ['Finnish', 'Swedish'], 'borders': ['Sweden', 'Norway', 'Russia'], 'government': 'Parliamentary republic', 'flag': {'colors': ['blue', 'white'], 'description': 'A blue Nordic cross on a white background'}, 'independenceDay': 'December 6, 1917'}


In [28]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

class Product(BaseModel):
    product_name: str = Field(description="Name of the product")
    product_details: str = Field(description="Details of the product")
    product_price: float = Field(description="Price of the product in eur")

parser = JsonOutputParser(pydantic_object=Product)

prompt = PromptTemplate(
    template="You are a e-commerces store chat agent. Answer in valid JSON. {format_instruction}\n\nQuestion: {input}",
    input_variables=["input"],
    partial_variables={"format_instruction": parser.get_format_instructions()}
)

chain = prompt | gptLlm | parser

try:
    result = chain.invoke({"input": "how is iphone 17"})
    print("Result:", result)
except Exception as e:
    print("Error invoking chain:", e)
    result = None

Result: {'product_name': 'iPhone 17', 'product_details': 'The iPhone 17 is our latest smartphone model featuring advanced technology, enhanced camera performance, and longer battery life. It is designed with a sleek new design and cutting edge features to improve your mobile experience.', 'product_price': 1099.0}
